In [36]:
import uproot
import awkward
import numpy as np
from scipy.sparse import csr_matrix

test = uproot.open('partGun_PDGid13_x2_E35.0To35.0_NTUP_1.root')['ana;1']['hgc;1']

In [178]:
#example of generating a simple adjacency graph for a single endcap
#this makes many assumptions like one particle per event per endcap
arrays = test.arrays(['simcluster_hits_indices','rechit_z'])
rechit_z = arrays['rechit_z']
sim_indices = awkward.fromiter(arrays['simcluster_hits_indices'])
valid_sim_indices = sim_indices[sim_indices > -1]

for ievt in range(rechit_z.size):
    distincts = valid_sim_indices[ievt].distincts()
    pos_z_mask = (rechit_z[ievt] > 0)
    rh_pos_z, = np.where(pos_z_mask)
    rh_neg_z, = np.where(~pos_z_mask)
    rh_idx_map = np.zeros(pos_z_mask.size)
    rh_idx_map[pos_z_mask] = np.arange(rh_pos_z.size)
    rh_idx_map[~pos_z_mask] = np.arange(rh_neg_z.size)
        
    pos_shape = (rh_pos_z.size,rh_pos_z.size)
    neg_shape = (rh_neg_z.size,rh_neg_z.size)
    
    adj_pos_z = csr_matrix(pos_shape,dtype=np.bool)
    adj_neg_z = csr_matrix(neg_shape,dtype=np.bool)    
    
    for iclu in range(distincts.size):
        ispos = np.all(rechit_z[ievt][valid_sim_indices[ievt][iclu]] > 0)
        print 'ispos',ispos
        
        row = rh_idx_map[distincts[iclu].i0]
        col = rh_idx_map[distincts[iclu].i1]
        data = np.ones(distincts[iclu].i0.size,dtype=np.bool)
                
        if ispos:
            mat = csr_matrix((data,(row,col)),shape=pos_shape,dtype=np.bool)
            adj_pos_z += mat
        else:
            mat = csr_matrix((data,(row,col)),shape=neg_shape,dtype=np.bool)
            adj_neg_z += mat
    
    print repr(adj_pos_z)
    print repr(adj_neg_z)


ispos False
ispos True
<15869x15869 sparse matrix of type '<type 'numpy.bool_'>'
	with 2346 stored elements in Compressed Sparse Row format>
<15575x15575 sparse matrix of type '<type 'numpy.bool_'>'
	with 1485 stored elements in Compressed Sparse Row format>
ispos False
ispos True
<15743x15743 sparse matrix of type '<type 'numpy.bool_'>'
	with 1176 stored elements in Compressed Sparse Row format>
<15849x15849 sparse matrix of type '<type 'numpy.bool_'>'
	with 1378 stored elements in Compressed Sparse Row format>
